In [1]:
from bs4 import BeautifulSoup, SoupStrainer
import urllib.request
import json
from datetime import datetime
import regex as re
import os
import pandas as pd

In [2]:
folders = ["data"]

for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

path_to_visited_urls = os.path.join("data", "visited_urls.json")

if not os.path.isfile(path_to_visited_urls):
    with open(path_to_visited_urls,"w") as file:
        json.dump([],file)

countries = {"Germany": "D",
             "Austria": "A",
             "Belgium" : "B",
             "Spain": "E",
             "France": "F",
             "Italy": "I",
             "Luxembourg": "L",
             "Netherlands": "NL"}

car_counter=1
cycle_counter=0

In [ ]:
while True:

    with open(path_to_visited_urls) as file:
        visited_urls = json.load(file)
    
    if len(visited_urls) > 100000:
        visited_urls = []
    
    multiple_cars_dict = {}
    
    cycle_counter+=1

    for country in countries:
        
        
        
        for page in range(1,21):

            try:
                url = 'https://www.autoscout24.de/lst/?sort=age&desc=1&ustate=N%2CU&size=20&page='+str(page)+ '&cy=' + countries[country] +'&atype=C&'
                only_a_tags = SoupStrainer("a")
                soup = BeautifulSoup(urllib.request.urlopen(url).read(),'lxml', parse_only=only_a_tags)
            except Exception as e:
                print("Übersicht: " + str(e) +" "*50, end="\r")
                pass

            for link in soup.find_all("a"):
                if r"/angebote/" in str(link.get("href")):
                    car_URLs.append(link.get("href"))

            car_URLs_unique = [car for car in list(set(car_URLs)) if car not in visited_urls]
            
            print(f'Lauf {cycle_counter} | {country} | Seite {page} | {len(car_URLs_unique)} neue URLs', end="\r")
        print("")
        if len(car_URLs_unique)>0:

            for URL in car_URLs_unique:
                print(f'Lauf {cycle_counter} | {country} | Auto {car_counter}'+' '*50, end="\r")
                try:
                    car_counter+=1

                    car_dict = {}
                    car_dict["country"] = country
                    car_dict["date"] = str(datetime.now())                    
                    car = BeautifulSoup(urllib.request.urlopen('https://www.autoscout24.de'+URL).read(),'lxml')
                    
                    for key, value in zip(car.find_all("dt"),car.find_all("dd")):
                        car_dict[key.text.replace("\n","")] = value.text.replace("\n","")

                    car_dict["haendler"] = car.find("div",attrs={"class":"cldt-vendor-contact-box",
                                                                 "data-vendor-type":"dealer"}) != None

                    car_dict["privat"] = car.find("div",attrs={"class":"cldt-vendor-contact-box",
                                                               "data-vendor-type":"privateseller"}) != None

                    car_dict["ort"] = car.find("div",attrs={"class":"sc-grid-col-12",
                                                            "data-item-name":"vendor-contact-city"}).text
                    
                    car_dict["price"] =  "".join(re.findall(r'[0-9]+',car.find("div",attrs={"class":"cldt-price"}).text))
                    
                    ausstattung = []

                    for i in car.find_all("div",attrs={"class":"cldt-equipment-block sc-grid-col-3 sc-grid-col-m-4 sc-grid-col-s-12 sc-pull-left"}):
                        for span in i.find_all("span"):
                            ausstattung.append(i.text)

                    ausstattung2 = []

                    for element in list(set(ausstattung)):
                        austattung_liste = element.split("\n")
                        ausstattung2.extend(austattung_liste)

                    car_dict["ausstattung_liste"] = sorted(list(set(ausstattung2)))

                    multiple_cars_dict[URL] = car_dict
                    visited_urls.append(URL)
                except Exception as e:
                    print("Detailseite: " + str(e) + " "*50)
                    pass
            print("")
        
        else:
            print("\U0001F634")
            sleep(60)
    
    if len(multiple_cars_dict)>0:
        df = pd.DataFrame(multiple_cars_dict).T
        df.to_csv("data/autos/"+re.sub("[.,:,-, ]","_",str(datetime.now()))+".csv",sep=";",index_label="url")
    else:
        print("Keine Daten")
    with open("data/visited/visited_urls.json", "w") as file:
        json.dump(visited_urls, file)

In [3]:
country = "Italy"
page=1
try:
    #url = 'https://www.autoscout24.it/lst?sort=age&desc=1&cy=I&atype=C&ustate=N%2CU&powertype=kw&search_id=aoxk2xf4jy&page=1'
    url = 'https://www.autoscout24.it/lst?sort=age&desc=1&ustate=N%2CU&size=20&page='+str(page)+ '&cy=' + countries[country] +'&atype=C'
    only_a_tags = SoupStrainer("a")
    soup = BeautifulSoup(urllib.request.urlopen(url).read(),'lxml', parse_only=only_a_tags)
except Exception as e:
    print("Error: " + str(e) +" "*50, end="\r")
    pass

pattern_manufacturer = re.compile(r'\"makeId\":(.*?),\"modelOrModelLineId\":(.*?),\"make\":"?(.*?)"?,\"model\":"?(.*?)"?,\"modelVersionInput\":"?(.*?)"?')
pattern_vendor_location = re.compile('"location":{"countryCode":"?(.*?)"?,"zip":"?(.*?)"?,"city":"?(.*?)"?,\"street\":"?(.*?)"?')

In [39]:
car_URLs = []
visited_urls = []
multiple_cars_dict = {}

for link in soup.find_all("a"):
    if r"/annunci/" in str(link.get("href")):
        car_URLs.append(link.get("href"))

car_URLs_unique = [car for car in list(set(car_URLs)) if car not in visited_urls]

In [47]:
car_URLs_unique

['/annunci/citroen-c3-1-1-ideal-benzina-nero-0e7105e7-af4e-439f-bf8b-6a3965c71ce5',
 '/annunci/fiat-panda-1-3-mjt-16v-diesel-bianco-1d5ff86d-6a53-4424-a250-4ea5cadc184b',
 '/annunci/ford-kuga-1-5-tdci-120cv-full-optional-da-vetrina-diesel-argento-c66288b5-091c-4a9a-ba46-a16cca37f700',
 '/annunci/bmw-x3-xdrive20d-4x4-diesel-bianco-a5ac7692-4f23-4bd9-9994-2870866315c5',
 '/annunci/fiat-panda-1-2-pop-69cv-benzina-7eb3049d-bfaa-43dc-a67e-c712719afaf9',
 '/annunci/mercedes-benz-180-a-d-automatic-sport-diesel-nero-6ca9480a-b6e2-4fd1-8b38-b8a6fb692d7e',
 '/annunci/mercedes-benz-180-a-d-automatic-sport-diesel-grigio-ebceaf64-632d-4dc5-bede-c1ef9608fc09',
 '/annunci/kia-rio-rio-1-2-cvvt-5p-active-benzina-bianco-f6cd9b8f-87db-4ee9-b1c8-3a3ad29005f3',
 '/annunci/hyundai-ix20-1-6-crdi-xpossible-diesel-nero-1c97d856-7c0f-4665-b41c-778117462a8d',
 '/annunci/alfa-romeo-giulietta-1-6-jtdm-120-cv-business-diesel-grigio-db6299ed-c52c-4368-a73f-f08f3becdfdf',
 '/annunci/ford-puma-1-0-ecoboost-hybrid-125-

In [45]:
%%timeit
for URL in car_URLs_unique[:10]:
    try:
        car_dict = {}
        car_dict["country"] = country
        car_dict["date"] = str(datetime.now())                    
        car = BeautifulSoup(urllib.request.urlopen('https://www.autoscout24.it'+URL).read(),'lxml')
        
        car_dict["makeId"], car_dict["modelOrModelLineId"], car_dict["maker"], car_dict["model"], car_dict["modelVersionInput"] = re.findall(pattern_manufacturer, car.find(id="__NEXT_DATA__").text)[0]
        for key, value in zip(car.find_all("dt"),car.find_all("dd")):
            if value.find_all("li"): #concatenate list of item
                car_dict[key.text.replace("\n","")] = ";".join([itm.text.replace("\n","") for itm in value.find_all("li")])
            else:
                car_dict[key.text.replace("\n","")] = value.text.replace("\n","")
                
        car_dict["dealer"] = car.find("div",attrs={"class":"cldt-vendor-contact-box","data-vendor-type":"dealer"}) != None
        car_dict["privateSeller"] = car.find("div",attrs={"class":"cldt-vendor-contact-box","data-vendor-type":"privateseller"}) != None
        car_dict["price"] =  car.find("span",attrs={"class":"StandardPrice_price__X_zzU"}).text
        
        car_dict["countryCode"], car_dict["zip"], car_dict["city"], car_dict["street"] = re.findall(pattern_vendor_location, car.find(id="__NEXT_DATA__").text)[0]        

        multiple_cars_dict[URL] = car_dict
        visited_urls.append(URL)
        
    except Exception as e:
        print("Error: " + str(e) +" "*10, end="\r")
        pass

6.63 s ± 434 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
if len(multiple_cars_dict)>0:
    df = pd.DataFrame(multiple_cars_dict).T
    #df.to_csv("data/autos/"+re.sub("[.,:,-, ]","_",str(datetime.now()))+".csv",sep=";",index_label="url")

In [44]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

,country,date,makeId,modelOrModelLineId,maker,model,modelVersionInput,Prezzo auto,Anticipo,Durata,Importo finanziato,Importo totale dovuto,TAEG,TAN Fisso,Rata,Carrozzeria,Tipo di veicolo,Trazione,Posti,Porte,Offerta n.,Usato garantito,Chilometraggio,Anno,Potenza,Tipo di cambio,Cilindrata,Marce,Carburante,Consumo di carburante,Emissioni CO₂,Classe emissioni,Comfort,Intrattenimento / Media,Sicurezza,Extra,Colore,Colore finiture interne,Materiale,Acconto,Costo del trasferimento,Costo di registrazione,"Totale, unico",Importo totale della locazione,Tipo di contratto,Chilometraggio p.a.,Fattore di locazione,Quota mensile,Più km costi,Risarcimento per meno km,Tasso di interesse effettivo,Interessi passivi pagati p.a.,Prestito netto,dealer,privateSeller,price,countryCode,zip,city,street,Per neopatentati,Tagliandi certificati,Veicolo non fumatori,Cilindri,Peso a vuoto,Versione per nazione,Ultimo tagliando,Proprietari,Tipo di vernice
/annunci/citroen-c3-1-1-ideal-benzina-nero-0e7105e7-af4e-439f-bf8b-6a3965c71ce5,Italy,2022-11-05 18:56:33.244043,21,18264,Citroen,C3,1.1 Ideal,"€ 4.290,-",-,36 Monate,"€ 4.300,-","€ 5.284,75","8,79 %","7,01 %","€ 85,15",Berlina,Usato,Anteriore,5,5,1236826,12 mesi,149.000 km,10/2009,44 kW (60 CV),Manuale,1.124 cm³,5,Benzina E10 91,"6 l/100 km (comb.)8 l/100 km (urbano)4,9 l/100...",139 g/km (comb.),Euro 5,Alzacristalli elettrici;Bracciolo;Climatizzato...,Autoradio;Computer di bordo;MP3,ABS;Airbag conducente;Airbag passeggero;Chiusu...,Ruota di riserva;Spoiler,Nero,Nero,Altro,"€ 4.750,-",Keine Angabe,Keine Angabe,"€ 4.750,-","€ 26.330,20",Kilometerleasing,10.000 km,"0,97","€ 599,45","€ 0,0776","€ 0,0518","6,16 %","5,99 % gebunden","€ 51.846,-",False,False,"€ 4.290,-5",IT,10148,Torino - TO,"Via Guglielmo Reiss Romoli, 92/A",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/annunci/fiat-panda-1-3-mjt-16v-diesel-bianco-1d5ff86d-6a53-4424-a250-4ea5cadc184b,Italy,2022-11-05 18:56:33.775805,28,1746,Fiat,Panda,1.3 mjt 16v,"€ 6.990,-",-,36 Monate,"€ 7.000,-","€ 8.419,90","7,81 %","6,60 %","€ 137,29",Furgoni/Van,Usato,Anteriore,2,5,NaN,12 mesi,50.000 km,03/2017,59 kW (80 CV),Manuale,1.248 cm³,5,Diesel,"3,6 l/100 km (comb.)4,3 l/100 km (urbano)3,2 l...",94 g/km (comb.),Euro 6d-TEMP,Alzacristalli elettrici;Climatizzatore;Start/S...,Bluetooth;CD;Computer di bordo;MP3;Sound syste...,ABS;Airbag conducente;Airbag laterali;Airbag p...,Cerchi in lega;Luce d'ambiente;Portapacchi;Tou...,Bianco,Nero,Stoffa,"€ 0,-",Enthalten,Enthalten,"€ 0,-","€ 7.884,-",Kilometerleasing,10.000 km,"2,78","€ 219,-","€ 0,0000","€ 0,0000","4,68 %","4,68 % gebunden","€ 17.980,-",False,False,"€ 6.990,-5",IT,10147,Torino - To,"Str. del Carossio, 11",Sì,Sì,Sì,4,1.124 kg,NaN,NaN,NaN,NaN
/annunci/ford-kuga-1-5-tdci-120cv-full-optional-da-vetrina-diesel-argento-c66288b5-091c-4a9a-ba46-a16cca37f700,Italy,2022-11-05 18:56:34.329435,29,19072,Ford,Kuga,1.5 TDCi 120cv *FULL OPTIONAL* da VETRINA,"€ 16.900,-",-,48 Monate,"€ 16.900,-","€ 20.411,40","7,98 %","7,31 %","€ 337,04",SUV/Fuoristrada/Pick-up,Usato,Anteriore,5,5,NaN,NaN,101.000 km,01/2018,88 kW (120 CV),Manuale,1.499 cm³,6,Diesel,"3,9 l/100 km (comb.)4,5 l/100 km (urbano)3,5 l...",115 g/km (comb.),Euro 6,Alzacristalli elettrici;Bracciolo;Chiusura cen...,Autoradio;Bluetooth;CD;Computer di bordo;MP3;S...,ABS;Airbag conducente;Airbag laterali;Airbag p...,"Cerchi in lega (17"");Portapacchi;Ruotino;Sedil...",Argento,Grigio,Stoffa,"€ 0,-","€ 799,-","€ 0,-","€ 799,-","€ 10.560,-",Kilometerleasing,10.000 km,"1,01","€ 220,-","€ 0,0559","€ 0,0250","4,97 %","4,97 % gebunden","€ 18.445,-",False,False,"€ 16.900,-1, 5",IT,72022,Latiano - Br,Via Piave 12,NaN,Sì,Sì,4,NaN,Italia,04/2022,1,Metallizzato


In [49]:
npages = 20
country = "Italy"

pattern_manufacturer = re.compile(r'\"makeId\":(.*?),\"modelOrModelLineId\":(.*?),\"make\":"?(.*?)"?,\"model\":"?(.*?)"?,\"modelVersionInput\":"?(.*?)"?')
pattern_vendor_location = re.compile('"location":{"countryCode":"?(.*?)"?,"zip":"?(.*?)"?,"city":"?(.*?)"?,\"street\":"?(.*?)"?')

visited_urls = []
car_URLs = []
multiple_cars_dict = {}

for page in range(1,npages+1):
        
    try:
        #url = 'https://www.autoscout24.it/lst?sort=age&desc=1&cy=I&atype=C&ustate=N%2CU&powertype=kw&search_id=aoxk2xf4jy&page=1'
        url = 'https://www.autoscout24.it/lst?sort=age&desc=1&ustate=N%2CU&size=20&page='+str(page)+ '&cy=' + countries[country] +'&atype=C'
        only_a_tags = SoupStrainer("a")
        soup = BeautifulSoup(urllib.request.urlopen(url).read(),'lxml', parse_only=only_a_tags)
    except Exception as e:
        print("Error: " + str(e) +" "*50, end="\r")
        pass

    for link in soup.find_all("a"):
        if r"/annunci/" in str(link.get("href")):
            car_URLs.append(link.get("href"))

car_URLs_unique = [car for car in list(set(car_URLs)) if car not in visited_urls]
print(f'{len(car_URLs_unique)} cars to be processed.')

for ii, URL in enumerate(car_URLs_unique):
    try:
        
        if((ii+1)%20==0): print(f'Processing car {ii+1}', end='\r')
        
        car_dict = {}
        car_dict["country"] = country
        car_dict["date"] = str(datetime.now())                    
        car = BeautifulSoup(urllib.request.urlopen('https://www.autoscout24.it'+URL).read(),'lxml')

        manufacturer_res = re.findall(pattern_manufacturer, car.find(id="__NEXT_DATA__").text)
        if manufacturer_res:
            car_dict["makeId"], car_dict["modelOrModelLineId"], car_dict["maker"], car_dict["model"], car_dict["modelVersionInput"] = manufacturer_res[0]
        else:
            car_dict["makeId"], car_dict["modelOrModelLineId"], car_dict["maker"], car_dict["model"], car_dict["modelVersionInput"] = ('NaN', 'NaN', 'NaN', 'NaN', 'NaN')
            print(f'Not found manufacturer in URL {URL}')

        for key, value in zip(car.find_all("dt"),car.find_all("dd")):
            if value.find_all("li"): #concatenate list of item
                car_dict[key.text.replace("\n","")] = ";".join([itm.text.replace("\n","") for itm in value.find_all("li")])
            else:
                car_dict[key.text.replace("\n","")] = value.text.replace("\n","")

        car_dict["dealer"] = car.find("div",attrs={"class":"cldt-vendor-contact-box","data-vendor-type":"dealer"}) != None
        car_dict["privateSeller"] = car.find("div",attrs={"class":"cldt-vendor-contact-box","data-vendor-type":"privateseller"}) != None
        car_dict["price"] =  car.find("span",attrs={"class":"StandardPrice_price__X_zzU"}).text

        location_res =  re.findall(pattern_vendor_location, car.find(id="__NEXT_DATA__").text)
        if location_res:
            car_dict["countryCode"], car_dict["zip"], car_dict["city"], car_dict["street"] = location_res[0]
        else:
            car_dict["countryCode"], car_dict["zip"], car_dict["city"], car_dict["street"] = ('NaN', 'NaN', 'NaN', 'NaN')
            print(f'Not found vendor location in URL {URL}')

        multiple_cars_dict[URL] = car_dict
        visited_urls.append(URL)

    except Exception as e:
        print(f"Error car {ii+1}: " + str(e) +" "*10)
        pass

400 cars to be processed.


In [50]:
if len(multiple_cars_dict)>0:
    df = pd.DataFrame(multiple_cars_dict).T
    df.to_csv("data/"+re.sub("[.,:,-, ]","_",str(datetime.now()))+".csv",sep=";",index_label="url")
    
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df.head())

,country,date,makeId,modelOrModelLineId,maker,model,modelVersionInput,Prezzo auto,Anticipo,Durata,Importo finanziato,Importo totale dovuto,TAEG,TAN Fisso,Rata,Carrozzeria,Tipo di veicolo,Posti,Porte,Chilometraggio,Anno,Tagliandi certificati,Veicolo non fumatori,Potenza,Tipo di cambio,Cilindrata,Marce,Cilindri,Carburante,Comfort,Intrattenimento / Media,Sicurezza,Extra,Colore,Tipo di vernice,Colore finiture interne,Materiale,Acconto,Costo del trasferimento,Costo di registrazione,"Totale, unico",Tipo di contratto,Chilometraggio p.a.,Quota mensile,Risarcimento per meno km,Tasso di interesse effettivo,Interessi passivi pagati p.a.,Prestito netto,dealer,privateSeller,price,countryCode,zip,city,street,Trazione,Per neopatentati,Proprietari,Peso a vuoto,Consumo di carburante,Emissioni CO₂,Importo totale della locazione,Fattore di locazione,Offerta n.,Anno di produzione,Classe emissioni,Colore specifico,Più km costi,Ultimo tagliando,Altre fonti energetiche,Revisione,Usato garantito,Rata finale,Disponibile da,Cambio cinghia distribuzione,Versione per nazione
/annunci/alfa-romeo-giulia-diesel-bianco-9645d1f5-aed4-4d81-9855-3787c2d5b358,Italy,2022-11-06 17:39:59.791116,6,18553,Alfa Romeo,Giulia,,"€ 28.500,-",-,30 Monate,"€ 28.500,-","€ 34.291,80","7,81 %","7,31 %","€ 568,38",Berlina,Usato,5,4,80.000 km,02/2018,Sì,Sì,110 kW (150 CV),Automatico,2.100 cm³,8,1,Diesel,Alzacristalli elettrici;Bracciolo;Climatizzato...,Autoradio;Bluetooth;CD;USB;Vivavoce,Airbag conducente;Airbag laterali;Airbag passe...,Leve al volante;Pneumatici da neve;Sedili spor...,Bianco,Metallizzato,Rosso,Pelle parziale,"€ 6.593,40",Keine Angabe,Keine Angabe,"€ 6.593,40",Kilometerleasing,0 km,"€ 160,-","€ 0,0000","5,31 %","5,31 % gebunden","€ 19.980,-",False,False,"€ 28.500,-",IT,20124,milano,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/annunci/peugeot-208-208-5p-1-6-bluehdi-allure-75cv-diesel-argento-635c597f-c0fa-4c23-af03-7693a11e0d52,Italy,2022-11-06 17:40:00.468326,55,20057,Peugeot,208,,"€ 10.000,-",-,36 Monate,"€ 10.000,-","€ 11.767,20","6,82 %","5,93 %","€ 192,97",Berlina,Usato,5,5,45.000 km,02/2016,NaN,Sì,55 kW (75 CV),Manuale,1.560 cm³,5,4,Diesel,Alzacristalli elettrici;Climatizzatore;Cruise ...,Autoradio;Autoradio digitale;Bluetooth;Compute...,ABS;Airbag conducente;Airbag laterali;Airbag p...,"Cerchi in lega (16"");Controllo vocale;Luce d'a...",Argento,Metallizzato,Nero,Stoffa,"€ 4.786,-",Keine Angabe,Enthalten,"€ 4.786,-",Kilometerleasing,15.000 km,"€ 231,-",NaN,"3,73 %","3,73 % gebunden","€ 23.930,-",False,False,"€ 10.000,-",IT,10142,Torino,,Anteriore,Sì,2,1.150 kg,"3,5 l/100 km (comb.)4,2 l/100 km (urbano)3 l/1...",90 g/km (comb.),"€ 25.666,97","0,9",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/annunci/alfa-romeo-1750-spider-duetto-osso-di-seppia-benzina-rosso-c0a2b489-6dc0-455a-9605-c39f2395a015,Italy,2022-11-06 17:40:01.289137,6,20940,Alfa Romeo,1750,,NaN,NaN,36 Monate,NaN,NaN,NaN,NaN,NaN,Cabrio,Usato,2,2,16.000 km,12/1968,NaN,NaN,96 kW (131 CV),Manuale,1.750 cm³,5,4,Benzina,NaN,NaN,NaN,NaN,Rosso,NaN,Nero,Pelle totale,"€ 3.653,50",Keine Angabe,Keine Angabe,"€ 3.653,50",Kilometerleasing,0 km,"€ 287,-","€ 0,0000","4,5 %","4,5 % gebunden","€ 32.589,99",False,False,"€ 63.500,-",IT,20900,monza,,Posteriore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
/annunci/ford-transit-custom-280-2-0-tdci-130-pc-furgone-trend-diesel-bianco-f35f14c4-9d90-4318-b0ea-3c608182e5ed,Italy,2022-11-06 17:40:01.865467,29,74248,Ford,Transit Custom,,"€ 17.991,-",-,36 Monate,"€ 18.000,-","€ 21.727,20","7,95 %","7,30 %","€ 358,97",Furgoni/Van,Usato,3,4,130.000 km,10/2019,NaN,NaN,96 kW (131 CV),Manuale,1.996 cm³,6,4,Diesel,Alzacristalli elettrici;Climatizzatore;Cruise ...,Autoradio;Computer di bordo,ABS;Airbag conducente;Chiusura centralizzata;C...,Cerchioni in acciaio,Bianco,NaN,NaN,Pelle scamosciata,"€ 11.999,85",Keine Angabe,Keine Angabe,"€ 11.999,85",Kilometerleasing,15.000 km,"€ 675,-","€ 0,0575","5,39 %",

In [78]:
URL = '/annunci/volkswagen-t-cross-t-cross-1-6-tdi-style-95cv-diesel-bianco-309e7d2b-357d-4a4f-9094-783f94b5f4e8'
car = BeautifulSoup(urllib.request.urlopen('https://www.autoscout24.it'+URL).read(),'lxml')

In [85]:
df = pd.read_csv('data/2022-11-06_23_00_07_064125.csv', sep=';', index_col='url')

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, /annunci/caterham-r300-superlight-sv-benzina-verde-cbaf5e51-788e-4d0b-a914-8aadfc0351e9 to /annunci/ferrari-gtc4-lusso-v12-valuto-permuta-benzina-bianco-49c8a302-6133-4c9f-9b5d-2fb9eb34de18
Data columns (total 83 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country                         400 non-null    object 
 1   date                            400 non-null    object 
 2   makeId                          400 non-null    int64  
 3   modelOrModelLineId              400 non-null    int64  
 4   maker                           400 non-null    object 
 5   model                           400 non-null    object 
 6   modelVersionInput               0 non-null      float64
 7   Carrozzeria                     400 non-null    object 
 8   Tipo di veicolo                 400 non-null    object 
 9   Chilometraggio                  396 non-nul

In [ ]:
df.index.str.strip('/annunci/')

In [77]:
df['street']

url
/annunci/caterham-r300-superlight-sv-benzina-verde-cbaf5e51-788e-4d0b-a914-8aadfc0351e9                           NaN
/annunci/citroen-c3-picasso-1-4-vti-limited-seduction-gpl-e5-gpl-grigio-e716415f-9dc4-4fff-a7c3-cbc58369c391      NaN
/annunci/bmw-x1-sdrive18d-xline-auto-my18-diesel-grigio-d73dbf88-d052-456d-acf4-335be61e42f8                      NaN
/annunci/opel-mokka-x-1-4-t-advance-gpl-tech-4x2-140cv-gpl-bianco-acf90c98-6344-4838-acd9-c9bfb1eb7916            NaN
/annunci/mercedes-benz-e-220-td-cdi-classic-diesel-argento-fa97ffc7-7419-439e-9250-54623e6f9031                   NaN
                                                                                                                   ..
/annunci/bmw-118-118d-3p-msport-diesel-bianco-fa2b2023-db9c-4cc8-b0bb-ef994435732c                                NaN
/annunci/volvo-xc60-xc60-2-0-d4-business-awd-geartronic-my18-diesel-bianco-32b68d38-4a0f-4ae0-938f-e76f2f1b5e0a   NaN
/annunci/fiat-500-500-1-0-hybrid-dolcevita-70cv-elet